In [162]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from sklearn.model_selection import train_test_split
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [163]:
data = pd.read_csv(r"C:\Users\amiya\Downloads\train.csv")

In [164]:
data

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive
...,...,...
765,NaN,NaN
766,NaN,NaN
767,NaN,NaN
768,NaN,NaN


In [165]:
data.dropna(inplace=True)

In [166]:
data

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive
...,...,...
594,4. GPT’s AI capabilities are also more accessi...,Positive
595,3. KeyboardX also provides a more secure envir...,Positive
596,2. KeyboardX allows users to quickly and easil...,Positive
597,"I asked ChatGPT why the app I’m working on, Ke...",Positive


In [167]:
text=data["Tweets"]

In [168]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Handle contractions
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespaces
    text = text.strip()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    # Join the words back into a sentence
    cleaned_text = ' '.join(filtered_text)
    return cleaned_text


In [169]:
data['cleaned_text'] = data['Tweets'].apply(clean_text)

In [170]:
labels=data["label"]

In [171]:
vocab_size = 10000 
embedding_dim = 16  
max_length = 50  
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [172]:
texts=data['cleaned_text']

In [173]:
# Tokenization
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [174]:
labels=data["label"]

In [175]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [176]:
# Build LSTM model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

In [177]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [178]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels for training set
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform labels for test set
y_test_encoded = label_encoder.transform(y_test)


In [179]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels for training set
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform labels for test set
y_test_encoded = label_encoder.transform(y_test)


In [180]:
num_epochs=10
history = model.fit(X_train, y_train_encoded, epochs=num_epochs, validation_data=(X_test, y_test_encoded), verbose=2)



Epoch 1/10
15/15 - 5s - loss: 1.0208 - accuracy: 0.6367 - val_loss: 0.9983 - val_accuracy: 0.5917 - 5s/epoch - 330ms/step
Epoch 2/10
15/15 - 1s - loss: 0.9133 - accuracy: 0.6409 - val_loss: 0.9335 - val_accuracy: 0.5917 - 598ms/epoch - 40ms/step
Epoch 3/10
15/15 - 1s - loss: 0.8885 - accuracy: 0.6409 - val_loss: 0.9305 - val_accuracy: 0.5917 - 645ms/epoch - 43ms/step
Epoch 4/10
15/15 - 1s - loss: 0.8662 - accuracy: 0.6409 - val_loss: 0.9341 - val_accuracy: 0.5917 - 692ms/epoch - 46ms/step
Epoch 5/10
15/15 - 1s - loss: 0.8328 - accuracy: 0.6409 - val_loss: 0.9496 - val_accuracy: 0.5917 - 614ms/epoch - 41ms/step
Epoch 6/10
15/15 - 1s - loss: 0.7051 - accuracy: 0.6576 - val_loss: 0.9273 - val_accuracy: 0.6083 - 597ms/epoch - 40ms/step
Epoch 7/10
15/15 - 1s - loss: 0.4365 - accuracy: 0.8225 - val_loss: 1.0574 - val_accuracy: 0.4917 - 598ms/epoch - 40ms/step
Epoch 8/10
15/15 - 1s - loss: 0.3538 - accuracy: 0.8518 - val_loss: 1.0027 - val_accuracy: 0.5833 - 693ms/epoch - 46ms/step
Epoch 9/10

In [181]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print("Test Accuracy:", test_acc)


4/4 [==============================] - 0s 11ms/step - loss: 1.3479 - accuracy: 0.6000
Test Accuracy: 0.6000000238418579


In [182]:

# Make predictions
predictions = model.predict(X_test)

4/4 [==============================] - 1s 10ms/step


In [183]:
def predict_sentiment(input_text):
    # Clean the input text
    cleaned_text = clean_text(input_text)
    
    # Tokenize and pad the input text
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    
    # Make prediction
    prediction = model.predict(padded_sequence)
    
    # Decode the prediction
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    
    return predicted_label[0]


In [184]:
predict_sentiment("The video is useful For those who want to use ChatGPT to understand and improve Phyton codes. #chatGPT  youtu.be/uZUcZEDdseY")

1/1 [==============================] - 0s 31ms/step


'Positive'